In [1]:
import ipsuite as ips

2023-05-08 10:02:03,127 (DEBUG): Welcome to IPS - the Interatomic Potential Suite!


In [3]:
mapping = ips.geometry.BarycenterMapping(data=None)
thermostat = ips.calculators.LagevinThermostat(temperature=300, friction=0.01, time_step=0.5)

with ips.Project(automatic_node_names=True, remove_existing_graph=True) as project:
    mol = ips.configuration_generation.SmilesToAtoms(smiles="O")

    packmol = ips.configuration_generation.Packmol(
        data=[mol.atoms], count=[10], density=997
    )

    cp2k = ips.calculators.CP2KSinglePoint(
        data=packmol, cp2k_files=["GTH_BASIS_SETS", "GTH_POTENTIALS", "dftd3.dat"]
    )

    geopt = ips.calculators.ASEGeoOpt(
        model=cp2k,
        data=packmol.atoms,
        optimizer="BFGS",
        run_kwargs={"fmax": 0.1},
    )

    md = ips.calculators.ASEMD(
        model=cp2k,
        thermostat=thermostat,
        data=geopt.atoms,
        data_id=-1,
        sampling_rate=1,
        dump_rate=1,
        steps=250,
    )  

project.run(repro=False)


Running DVC command: 'stage add --name SmilesToAtoms --force ...'
Running DVC command: 'stage add --name Packmol --force ...'
Running DVC command: 'stage add --name CP2KSinglePoint --force ...'
Running DVC command: 'stage add --name ASEGeoOpt --force ...'
Running DVC command: 'stage add --name ASEMD --force ...'
Running DVC command: 'stage add --name ASEMD_thermostat --force ...'
Could not create .gitignore entry in /ssd/fzills/tests/ipsuite/IPS-Example00/nodes/ASEMD_thermostat/.gitignore. DVC will attempt to create .gitignore entry again when the stage is run.
